In [260]:
import wfdb
from FeatureExtraction_NB import processing,process_signal,process_qrs
import matplotlib.pyplot as plt
import pywt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [261]:
fs = 1000

patient_1 = wfdb.rdrecord('..//01.Dataset/117/s0291lre', channels=[1])
patient_2 = wfdb.rdrecord('..//01.Dataset/116/s0302lre', channels=[1])
patient_3 = wfdb.rdrecord('..//01.Dataset/105/s0303lre', channels=[1])
patient_4 = wfdb.rdrecord('..//01.Dataset/104/s0306lre', channels=[1])
patient_5 = wfdb.rdrecord('..//01.Dataset/122/s0312lre', channels=[1])
patient_6 = wfdb.rdrecord('..//01.Dataset/121/s0311lre', channels=[1])
patient_7 = wfdb.rdrecord('..//01.Dataset/150/s0287lre', channels=[1])

signal_1,_,_ = processing(patient_1.p_signal[: ,0])
signal_2 ,_,_= processing(patient_2.p_signal[:, 0])
signal_3 ,_,_= processing(patient_3.p_signal[:, 0])
signal_4,_,_ = processing(patient_4.p_signal[:, 0])
signal_5 ,_,_= processing(patient_5.p_signal[:, 0])
signal_6,_,_ = processing(patient_6.p_signal[:, 0])
signal_7 ,_,_= processing(patient_7.p_signal[:, 0])

time = len(signal_1)/ fs


In [262]:
def get_Rs(signal):
    denoised_signal, y_lfiltered, window_smoothed_signal = processing(signal)
    qx, qy, sx, sy = process_signal(denoised_signal, y_lfiltered)
    qx, qy, sx, sy, Rx, Ry, qrs_on_x, qrs_on_y, qrs_off_x, qrs_off_y = process_qrs(denoised_signal, y_lfiltered, window_smoothed_signal, qx, qy, sx, sy)
    return denoised_signal,Rx

In [263]:
def non_fiducial_features_bonus(signal):
    denoised_signal,Rx = get_Rs(signal)
    nonFiducials_for_Full_signal_1 = []

    for i in range(1,len(Rx)-1):
        RR_previous = Rx[i]-Rx[i-1]
        RR_next = Rx[i+1]-Rx[i]

        nonFiducial=[]

        after_Rpeak = int (2/3*((RR_previous+RR_next)/2))

        for x in range( int(Rx[i]) , int(Rx[i])+after_Rpeak) :
            nonFiducial.append( denoised_signal[x])

        Before_Rpeak = int (1/3*((RR_previous+RR_next)/2))

        for j in range(Before_Rpeak+int(Rx[i-1]) , int(Rx[i]) ):
            nonFiducial.append( denoised_signal[j])
            
        nonFiducials_for_Full_signal_1.append(nonFiducial)
        
    list_of_non_fiducial_features_1 = []
    for i in range(len(nonFiducials_for_Full_signal_1)):
        # Define the mother wavelet
        wavelet = pywt.Wavelet('db4')

        # Define the number of levels for decomposition
        decomp_levels = 5

        # Define the list of ECG signals
        ecg_segments = np.array(nonFiducials_for_Full_signal_1[i])  # Example data

        # Decompose the signal
        decomp = pywt.wavedec(ecg_segments, wavelet, level=decomp_levels)

        CA5,CD5,CD4,CD3,CD2,CD1 =decomp 

        #only the coefficients of ECG band (1-40) use them as a feature 
        non_fiducial_feature = CA5[:40]
        non_fiducial_feature = list(non_fiducial_feature)
        if len(non_fiducial_feature) < 40:
            for i in range(40-len(non_fiducial_feature)):
                non_fiducial_feature.append(0)
        list_of_non_fiducial_features_1.append(np.array(non_fiducial_feature))
    return list_of_non_fiducial_features_1

In [264]:
non_fid1 = non_fiducial_features_bonus(signal_1)
non_fid2 = non_fiducial_features_bonus(signal_2)
non_fid3 = non_fiducial_features_bonus(signal_3)
non_fid4 = non_fiducial_features_bonus(signal_4)
non_fid5 = non_fiducial_features_bonus(signal_5)
non_fid6 = non_fiducial_features_bonus(signal_6)
non_fid7 = non_fiducial_features_bonus(signal_7)

In [265]:
def dfs(non_fid, label):
    df = pd.DataFrame({})
    for i in range(len(non_fid)):
        df[i] = non_fid[i]
    df = df.T
    df['label'] = label
    return df

In [266]:
df1 = dfs(non_fid1, 1)
df2 = dfs(non_fid2, 2)
df3 = dfs(non_fid3, 3)
df4 = dfs(non_fid4, 4)
df5 = dfs(non_fid5, 5)
df6 = dfs(non_fid6, 6)
df7 = dfs(non_fid7, 7)

In [267]:
Data = pd.concat([df1, df2, df3, df4,df5,df6], ignore_index=True)

In [268]:
Data

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,5.047771,5.091184,5.078735,4.967018,5.657203,3.025052,-0.591736,-0.371068,-0.333918,-0.288880,...,-0.114856,-0.107610,-0.143382,-0.222201,-0.089995,-0.064123,-0.236230,-0.540547,-0.299259,1
1,5.077009,5.120414,5.108616,4.995187,5.691347,3.041182,-0.564897,-0.362978,-0.357267,-0.285199,...,-0.121520,-0.113629,-0.147046,-0.141902,-0.192655,0.039904,-0.134352,-0.406173,-0.504051,1
2,4.978195,5.024929,5.010683,4.897055,5.599241,2.929636,-0.558846,-0.394277,-0.348331,-0.285688,...,-0.103941,-0.135927,-0.117231,-0.131336,-0.165811,0.000020,-0.019494,-0.227953,-0.579532,1
3,4.991281,5.034845,5.021516,4.913158,5.586506,3.017311,-0.523826,-0.355513,-0.319881,-0.242623,...,-0.157440,-0.190145,-0.196086,-0.234971,-0.047890,-0.083108,-0.266474,-0.584379,-0.322452,1
4,4.953668,5.001299,4.986493,4.872775,5.576415,2.900661,-0.575943,-0.376332,-0.353331,-0.253833,...,-0.141000,-0.166454,-0.189164,-0.236909,-0.052670,-0.049229,-0.234406,-0.573441,-0.366860,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,2.628148,2.660637,2.656253,2.562271,3.112962,1.030734,-0.724105,-0.333350,-0.271138,-0.197525,...,-0.052540,-0.307682,-0.542824,0.180402,2.890553,0.000000,0.000000,0.000000,0.000000,6
774,2.554109,2.590650,2.584285,2.488496,3.049859,0.930704,-0.727707,-0.313992,-0.239460,-0.178078,...,0.164970,-0.288417,-0.377252,-0.456747,1.997903,2.803819,0.000000,0.000000,0.000000,6
775,2.486747,2.518121,2.513596,2.424282,2.948648,0.964020,-0.727742,-0.334394,-0.242475,-0.148783,...,-0.090706,-0.187432,0.013737,0.562894,0.173418,-0.281696,-0.326818,-0.406287,1.888365,6
776,2.779999,2.812154,2.807831,2.715321,3.256464,1.210348,-0.689821,-0.372212,-0.289017,-0.182717,...,-0.107545,-0.178600,0.023994,0.559658,0.177610,-0.267583,-0.310984,-0.522411,1.755123,6


In [269]:
X = Data.drop('label', axis=1)
y = Data['label']

# Model

In [270]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [271]:
lr = LogisticRegression(random_state=0).fit(X, y)

# Make predictions on the test set
y_pred = lr.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [272]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [273]:
clf = OneVsRestClassifier(SVC()).fit(X, y)

In [278]:
X_test = df7.drop('label', axis=1)
y_test = df7['label']

y_pred = clf.decision_function(X_test)

In [279]:
y_pred_o = clf.predict(X_test)
y_pred_o

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int64)

In [282]:
cnt = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0 , 0:0}

for i in range(len(y_pred)):
    for indx in range(0,6):
        if y_pred[i][indx] < 1.3:
            print(y_pred[i][indx])
            if y_pred[i][indx] > 0.8:
                cnt[indx+1] += 1
        else:
            cnt[0] += 1

-2.44652808778973
-3.8442473576221623
-2.118412279802101
-3.3115695791477036
0.0786406706059064
-2.3571538623253225
-3.850101846297312
-2.2147656195884973
-3.3423794548897834
-0.053963282231757725
-2.235561595079938
-3.8180135736076255
-2.346580506636581
-3.3952740925160096
-0.2601318437532265
-2.142387285321059
-3.8156985739551086
-2.296589962714057
-3.4330458761275855
-0.2673543368655211
-2.256093009492609
-3.830015768046863
-2.2641360034707825
-3.3887074495811182
-0.09971627640288405
-2.3682525961075296
-3.831427123413754
-2.191377936508773
-3.3258082483081695
0.06938520321081454
-2.4551844229993947
-3.8344648360335265
-2.1080308950016695
1.275494743618165
-3.2908291122225553
0.15234055188521078
-2.2262809355802227
-3.8384868554762566
-2.297177925542883
-3.3837130997711125
-0.11912284932790618
-2.141422356595818
-3.8312270014596854
-2.370206434996339
-3.40387144954216
-0.18321898789672275
-1.972342910506989
-3.8340735128882573
-2.3523150876522294
-3.422861010261437
-0.29597469729664

In [283]:
cnt

{1: 0, 2: 0, 3: 0, 4: 37, 5: 0, 6: 0, 0: 83}